In [30]:
import os
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Dataset parameters
dataset_path = 'C:/Users/nisht/Project/ISL_Dataset'  # Replace with your dataset path
IMG_SIZE = 64
BATCH_SIZE = 32

# Data loading and preprocessing
def load_data(dataset_path, img_size=IMG_SIZE):
    images = []
    labels = []
    
    for label_dir in os.listdir(dataset_path):
        label_path = os.path.join(dataset_path, label_dir)
        if os.path.isdir(label_path):
            for img_file in os.listdir(label_path):
                img_path = os.path.join(label_path, img_file)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (img_size, img_size))  # Resize image
                images.append(img)
                labels.append(label_dir)
    
    images = np.array(images) / 255.0  # Normalize images
    labels = np.array(pd.factorize(labels)[0])  # Convert labels to integers
    return images, labels

images, labels = load_data(dataset_path)

# Split data
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)


C:\Users\nisht\AppData\Local\Temp\ipykernel_29216\2424267059.py:30: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  labels = np.array(pd.factorize(labels)[0])  # Convert labels to integers


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def build_cnn():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(np.unique(labels)), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train CNN model
cnn_model = build_cnn()
cnn_model.fit(train_generator, epochs=20, validation_data=(X_test, y_test))
y_pred_cnn = np.argmax(cnn_model.predict(X_test), axis=1)
print(classification_report(y_test, y_pred_cnn))
cnn_model.save('cnn_model.h5')

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

def build_vgg16():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    for layer in base_model.layers:
        layer.trainable = False
    model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(np.unique(labels)), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train VGG16 model
vgg16_model = build_vgg16()
vgg16_model.fit(train_generator, epochs=20, validation_data=(X_test, y_test))
y_pred_vgg16 = np.argmax(vgg16_model.predict(X_test), axis=1)
print(classification_report(y_test, y_pred_vgg16))
vgg16_model.save('vgg16_model.h5')

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

def build_resnet50():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    for layer in base_model.layers:
        layer.trainable = False
    model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(np.unique(labels)), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train ResNet50 model
resnet50_model = build_resnet50()
resnet50_model.fit(train_generator, epochs=20, validation_data=(X_test, y_test))
y_pred_resnet50 = np.argmax(resnet50_model.predict(X_test), axis=1)
print(classification_report(y_test, y_pred_resnet50))
resnet50_model.save('resnet50_model.h5')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D
import numpy as np

# Preparing the data for ConvLSTM
# ConvLSTM expects a 5D tensor: (samples, timesteps, rows, cols, channels)
# Here, we simulate "timesteps" by adding a dummy dimension

def prepare_data_for_convlstm(images, timesteps=1):
    # Adding a time dimension for ConvLSTM (assuming static images treated as one timestep each)
    images = np.expand_dims(images, axis=1)  # Shape: (samples, timesteps, rows, cols, channels)
    return images

X_train_lstm = prepare_data_for_convlstm(X_train)
X_test_lstm = prepare_data_for_convlstm(X_test)

# Model Definition
def build_convlstm(input_shape):
    model = Sequential([
        # ConvLSTM layer
        ConvLSTM2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=input_shape, return_sequences=False),
        BatchNormalization(),
        
        # Fully connected layers
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(np.unique(labels)), activation='softmax')  # Output layer with softmax
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Build the model
input_shape = (X_train_lstm.shape[1], IMG_SIZE, IMG_SIZE, 3)  # (timesteps, rows, cols, channels)
convlstm_model = build_convlstm(input_shape)

# Train the model
history = convlstm_model.fit(
    X_train_lstm, y_train,
    epochs=20,
    batch_size=BATCH_SIZE,
    validation_data=(X_test_lstm, y_test)
)

# Evaluate the model
y_pred_lstm = np.argmax(convlstm_model.predict(X_test_lstm), axis=1)
print("Classification Report:\n", classification_report(y_test, y_pred_lstm))
print("Accuracy Score:", accuracy_score(y_test, y_pred_lstm))

# Save the model
convlstm_model.save('convlstm_model.h5')

In [ ]:
# Summarizing model performance
model_reports = {
    "CNN": classification_report(y_test, y_pred_cnn, output_dict=True),
    "VGG16": classification_report(y_test, y_pred_vgg16, output_dict=True),
    "ResNet50": classification_report(y_test, y_pred_resnet50, output_dict=True),
    "ConvLSTM": classification_report(y_test, y_pred_lstm, output_dict=True),
}

best_model_name = max(model_reports, key=lambda x: model_reports[x]['accuracy'])
print(f"The best model is {best_model_name}.")